 [【中文分词系列】 3. 字标注法与HMM模型 - 科学空间|Scientific Spaces](https://kexue.fm/archives/3922#Python%E5%AE%9E%E7%8E%B0)


In [1]:
#export
import sys
if __name__ == '__main__': sys.path.append('..')
import exp.common as common

In [3]:
common.IN_JUPYTER

True

In [4]:
#test_export

# 准备发射概率
# https://github.com/fxsjy/jieba/blob/master/jieba/dict.txt

from collections import Counter
from functools import reduce

BMES = 'BMES'
B,M,E,S = list(BMES)

def each_line(dictionary, line):
    
    # 词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒
    line = line.split(' ')
    word = line[0]
    
    freq = line[1] if len(line) > 1 else '1'
    freq = int(freq) if freq.isdigit() else 1
    
    assert word, "word cant be empty str"
    
    if len(word) == 1:
        dictionary[S][word] += freq
    else:
        dictionary[B][word[0]] += freq
        dictionary[E][word[-1]] += freq
        for w in word[1:-1]: dictionary[M][w] += freq
    
    return dictionary

with open('../data/HMM/dict.txt', 'r', encoding='UTF-8') as f:
    
    dictionary = reduce(each_line, f.readlines(), dict(map(lambda t:(t, Counter()), BMES)))

dictionary_sum = dict(map(lambda tp:(tp[0], sum(tp[1].values())), dictionary.items()))

In [5]:
#test_export
import sure

In [33]:
#test_export
import math
import numpy as np
from scipy.special import logsumexp

# 准备转移概率
# https://raw.githubusercontent.com/fxsjy/jieba/master/jieba/finalseg/prob_trans.py

# 对数概率
from data.HMM.prob_trans import P as trans
from data.HMM.prob_start import P as start
for k,v in trans.items():
    float(logsumexp(list(v.values()))).should.eql(.0, 1e-8) # log(1) = 0
    
    for k1, v1 in v.items():
        print(f'{k}->{k1}:{v1}\t {math.exp(v1)}')
        

B->E:-0.51082562376599	 0.6000000000000004
B->M:-0.916290731874155	 0.4
E->B:-0.5897149736854513	 0.5544853051164425
E->S:-0.8085250474669937	 0.44551469488355755
M->E:-0.33344856811948514	 0.7164487459986911
M->M:-1.2603623820268226	 0.2835512540013088
S->B:-0.7211965654669841	 0.48617017333894563
S->S:-0.6658631448798212	 0.5138298266610544


In [32]:
#export
import operator
def get_max_tuple(d):
    assert isinstance(d, dict)
    return max(d.items(), key=operator.itemgetter(1))
    

def viterbi(nodes,trans_p, initial_state = None, return_max = True, start_p = None):
    """
    nodes: array, [{"<TAG>": <float>}]
    trans_p: dict, {"<TAG_A>": {"TAG_B": <float>}}, TAG_A -> TAG-B
    return_max=True: bool, set to False if your want to avoid ill endding tag and find your own.
    initial_state=None: None or list to avoid impossible starting tags
    start_p: P(o1), may be nessary
    all float number in log form
    """
    if initial_state is None:
        initial_state = nodes[0]
        nodes = nodes[1:]
        
    if start_p:
        for k in initial_state:
            if k in start_p:
                initial_state[k] *= start_p[k]
        
    path_score = initial_state
    
    
    for node in nodes:
        new_path_score = {}
        for t,v in node.items():
            t_new_path_score = {}
            
            for path, score in path_score.items():
                
                last_t = path[-1]
                new_path = path + t
                
                tran = trans_p[last_t].get(t, None)
                if tran is None:
                # 不合法的转移
                    pass
                else :
                    t_new_path_score[new_path] = score + tran + v
                
            new_path_score.__setitem__(*get_max_tuple(t_new_path_score))
        path_score = new_path_score
    return get_max_tuple(path_score) if return_max else path_score

In [29]:
#export
def decode_seq_tags(seq, tags, splitter_tags=['S', 'E'], word_spliter='|'):
    ret = ''
    splitter_tags = set(splitter_tags)
    for c, t in zip(seq,tags):
        
        if t in splitter_tags:
            ret += c
            ret += word_spliter
        else :
            ret += c
    
    if tags[-1] in splitter_tags:
        return ret[:-1]
            
    return ret
        

In [14]:
#test_export
decode_seq_tags('abcd', "BMES").should.equal("abc|d")

True

In [15]:
#export

from math import log
def get_cal_nodes_by_dict(dictionary):
    """
    dictionary: {<TAG>:{<char>:freq}}
    """
    
    dictionary_sum = dict(map(lambda tp:(tp[0], sum(tp[1].values())), dictionary.items()))
    def cal_nodes(sentence):
        """
        sentence: str, 
        # 加1平滑
        return: [c在给定<TAG>下对数概率{<TAG>:float} for c in sentence]
        """
        nodes = []
        for w in s:
            d = {}
            for (t, v) in dictionary.items():
                # 加1平滑
                d[t] = log(v.get(w, 0)+1) - log(dictionary_sum[t])
            nodes.append(d)
        return nodes
    return cal_nodes
#         print(t, v[w], v[w]/dictionary_sum[t], log(v[w]) - log(dictionary_sum[t]))



In [16]:
#test_export

cal_nodes = get_cal_nodes_by_dict(dictionary)

def tester(s):
    nodes = cal_nodes(s)
    tags, _ = viterbi(nodes, trans)
    return decode_seq_tags(s, tags, word_spliter=' ')

In [36]:
#test_export

s = "我是中国人"
nodes = cal_nodes(s)
tags, _ = viterbi(nodes, trans, start_p=start)
decode_seq_tags(s, tags).should.eql('我|是|中国|人')

True

In [23]:
#test_export

s = "我是中国人"
tester(s).should.eql("我 是 中国人")

True

In [24]:
#test_export

s = "李想是一个好孩子"
tester(s).should.eql("李想 是 一个 好 孩子")

True

In [25]:
#test_export

s = "小明硕士毕业于中国科学院计算所"
tester(s).should.eql('小明 硕士 毕业 于 中国 科学院 计算 所')

True

In [26]:
#test_export

s = """
中国是世界上历史最悠久的国家之一，有着光辉灿烂的文化和光荣的革命传统 [3]  ，世界遗产数量全球领先 [10]  。1949年新中国成立后，进入社会主义革命和建设时期，1956年实现向社会主义过渡，此后社会主义建设在探索中曲折发展 [11]  。“文化大革命”结束后实行改革开放，沿着中国特色社会主义道路，集中力量进行社会主义现代化建设 [3]  。经过长期努力，中国特色社会主义进入了新时代。 [12]  
"""
s = s.strip()
import re
s = re.sub(r"\s+", '', s, flags = re.MULTILINE)
s = re.sub(r"\[\d+\]", '', s, flags = re.MULTILINE)
tester(s)

'中国 是 世界 上 历史 最 悠久 的 国家 之 一，有 着 光辉 灿烂 的 文化 和 光荣 的 革命 传统 ，世界 遗产 数量 全球领 先。19 49年 新 中国 成立 后 ， 进入 社会 主义 革命 和 建设 时期 ， 1956年 实现 向 社会 主义 过 渡，此 后 社会 主义 建设 在 探索 中曲折 发展 。“文化 大革命 ” 结束 后 实行 改革 开放， 沿着 中国 特色 社会 主义 道 路，集 中力量 进行 社会 主义 现代化 建设 。 经过 长期 努力 ， 中国 特色 社会 主义 进入 了 新 时代。'

In [ ]:
common.save_and_export_notebook('HMM')

<IPython.core.display.Javascript object>

Converted HMM.ipynb to ..\exp\HMM.py
Converted HMM.ipynb to ..\test\test_HMM.py


In [70]:
not None

True